# 01 - Data Quality Assessment

Comprehensive data quality report for all 9 tables in the Olist e-commerce dataset.

**Tables analyzed:**
- orders, order_items, order_payments, order_reviews
- products, customers, sellers, geolocation
- product_category_translation

## Setup

In [ ]:
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect('../data/olist_ecommerce.db')

# List all tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tables in database:")
print(tables['name'].tolist())

## Data Quality Report Function

In [ ]:
def data_quality_report(df, name):
    print(f"\n{'='*60}")
    print(f"DATA QUALITY REPORT: {name}")
    print(f"{'='*60}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"\nNull counts:")
    nulls = df.isnull().sum()
    print(nulls[nulls > 0] if nulls.sum() > 0 else "No nulls found")
    print(f"\nNull percentages:")
    null_pct = (df.isnull().sum() / len(df) * 100).round(2)
    print(null_pct[null_pct > 0] if null_pct.sum() > 0 else "No nulls")
    print(f"\nDuplicates: {df.duplicated().sum():,}")
    print(f"\nData types:\n{df.dtypes}")

## Load All Tables

In [ ]:
# Load all tables into dataframes
orders = pd.read_sql("SELECT * FROM orders", conn)
order_items = pd.read_sql("SELECT * FROM order_items", conn)
order_payments = pd.read_sql("SELECT * FROM order_payments", conn)
order_reviews = pd.read_sql("SELECT * FROM order_reviews", conn)
products = pd.read_sql("SELECT * FROM products", conn)
customers = pd.read_sql("SELECT * FROM customers", conn)
sellers = pd.read_sql("SELECT * FROM sellers", conn)
geolocation = pd.read_sql("SELECT * FROM geolocation", conn)
product_category_translation = pd.read_sql("SELECT * FROM product_category_translation", conn)

print("All tables loaded successfully!")

## Data Quality Reports

### Orders Table

In [ ]:
data_quality_report(orders, "orders")

### Order Items Table

In [ ]:
data_quality_report(order_items, "order_items")

### Order Payments Table

In [ ]:
data_quality_report(order_payments, "order_payments")

### Order Reviews Table

In [ ]:
data_quality_report(order_reviews, "order_reviews")

### Products Table

In [ ]:
data_quality_report(products, "products")

### Customers Table

In [ ]:
data_quality_report(customers, "customers")

### Sellers Table

In [ ]:
data_quality_report(sellers, "sellers")

### Geolocation Table

In [ ]:
data_quality_report(geolocation, "geolocation")

### Product Category Translation Table

In [ ]:
data_quality_report(product_category_translation, "product_category_translation")

## Key Findings Summary

### Null Values

| Table | Columns with Nulls | Notes |
|-------|-------------------|-------|
| **orders** | `order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date` | Expected for pending/in-transit orders |
| **order_reviews** | `review_comment_title`, `review_comment_message` | Optional fields - customers can leave rating without comments |
| **products** | `product_category_name`, `product_name_length`, `product_description_length`, `product_photos_qty`, `product_weight_g`, `product_length_cm`, `product_height_cm`, `product_width_cm` | Some products missing category or dimension data |

### Duplicate Issues

| Table | Duplicate Count | Concern Level |
|-------|----------------|---------------|
| **geolocation** | High | Expected - multiple entries per zip code (different lat/lng coordinates within same area) |
| Others | Low/None | No major concerns |

### Data Type Issues

| Table | Columns | Current Type | Should Be |
|-------|---------|-------------|-----------|
| **orders** | All timestamp columns | `object` (string) | `datetime64` |
| **order_items** | `shipping_limit_date` | `object` (string) | `datetime64` |
| **order_reviews** | `review_creation_date`, `review_answer_timestamp` | `object` (string) | `datetime64` |

### Important: Customer ID vs Customer Unique ID

The `customers` table has two ID columns:
- **`customer_id`**: Unique per order - used to join with orders table
- **`customer_unique_id`**: Unique per actual customer - use this to count unique customers and track repeat purchases

This is important for customer analytics - always use `customer_unique_id` when counting unique customers or analyzing customer behavior over time.

### Recommendations for Data Cleaning

1. **Convert timestamp columns** to proper datetime format
2. **Handle geolocation duplicates** by aggregating (e.g., take mean lat/lng per zip code) when joining
3. **Handle nulls appropriately**:
   - Delivery timestamps: Keep as null (represents status)
   - Review comments: Keep as null (optional fields)
   - Product dimensions: May need imputation or exclusion for analyses requiring this data

---

## Data Cleaning Implementation

Now that we've assessed data quality, let's implement the cleaning steps.

### Step 1: Timestamp Conversion

Convert all timestamp columns in the orders table from strings to datetime objects for time-series analysis.

In [ ]:
# Convert timestamp columns to datetime
timestamp_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in timestamp_columns:
    orders[col] = pd.to_datetime(orders[col])

print("Timestamp columns converted:")
print(orders[timestamp_columns].dtypes)

### Step 2: Handle Null Review Comments

**Decision:** Keep null `review_comment_title` and `review_comment_message` as-is.

**Reasoning:** Missing text does not equal a missing review. The `review_score` is the primary metric for sentiment analysis. Many customers leave a star rating without writing a text comment, and this is valid review behavior.

In [ ]:
# Verify review comments null counts - no action needed, just documenting
print("Review comments null counts (keeping as-is):")
print(f"  review_comment_title: {order_reviews['review_comment_title'].isnull().sum():,} nulls")
print(f"  review_comment_message: {order_reviews['review_comment_message'].isnull().sum():,} nulls")
print(f"\nTotal reviews: {len(order_reviews):,}")
print(f"Reviews with scores (all): {order_reviews['review_score'].notna().sum():,}")

### Step 3: Deduplicate Geolocation

The geolocation table has many duplicates per zip code (multiple lat/lng readings for the same postal code). For analysis purposes, we keep the first occurrence per zip code.

In [ ]:
# Deduplicate geolocation - keep first occurrence per zip code
print(f"Original geolocation rows: {len(geolocation):,}")
print(f"Unique zip codes: {geolocation['geolocation_zip_code_prefix'].nunique():,}")

geolocation_clean = geolocation.drop_duplicates(
    subset=['geolocation_zip_code_prefix'], 
    keep='first'
)

print(f"\nAfter deduplication: {len(geolocation_clean):,} rows")
print(f"Rows removed: {len(geolocation) - len(geolocation_clean):,}")

### Step 4: Handle Null Product Categories

Some products have null `product_category_name`. We fill these with 'other' to maintain data completeness for category-based analyses.

In [ ]:
# Fill null product categories with 'other'
print(f"Products with null category: {products['product_category_name'].isnull().sum():,}")

products['product_category_name'] = products['product_category_name'].fillna('other')

print(f"\nAfter filling nulls: {products['product_category_name'].isnull().sum()} nulls")
print(f"Products now categorized as 'other': {(products['product_category_name'] == 'other').sum():,}")

### Step 5: Filter Delivered Orders

For most analyses, we focus on completed transactions. We create a filtered dataset of delivered orders only.

In [ ]:
# Filter to delivered orders only
print("Order status distribution:")
print(orders['order_status'].value_counts())

orders_delivered = orders[orders['order_status'] == 'delivered']

print(f"\nTotal orders: {len(orders):,}")
print(f"Delivered orders: {len(orders_delivered):,}")
print(f"Percentage delivered: {len(orders_delivered)/len(orders)*100:.1f}%")

### Step 6: Understanding customer_id vs customer_unique_id

The customers table contains two important ID columns that serve different purposes:

| Column | Description | Use Case |
|--------|-------------|----------|
| **customer_id** | Unique identifier per order | Use for joining with orders table. Each order has a unique customer_id. |
| **customer_unique_id** | Unique identifier per person | Use for customer analytics: counting unique customers, repeat purchase analysis, customer lifetime value. |

**Why this matters:** A single person (customer_unique_id) can place multiple orders, each with a different customer_id. If you count customer_id, you count orders. If you count customer_unique_id, you count actual customers.

In [ ]:
# Demonstrate the difference between customer_id and customer_unique_id
print("Customer ID analysis:")
print(f"  Total customer_id entries: {len(customers):,}")
print(f"  Unique customer_id: {customers['customer_id'].nunique():,}")
print(f"  Unique customer_unique_id: {customers['customer_unique_id'].nunique():,}")
print(f"\nThis means {len(customers) - customers['customer_unique_id'].nunique():,} repeat customers exist in the dataset.")

---

## Data Cleaning Summary

| Issue | Table | Resolution | Reasoning |
|-------|-------|------------|-----------|
| Timestamps as strings | orders | Convert to datetime | Required for time-series analysis |
| Null review comments | order_reviews | Keep nulls | Missing text does not equal missing review; review_score is the primary metric |
| Duplicate geolocations | geolocation | Keep first per zip | Multiple lat/lng readings per zip code |
| Null category names | products | Fill with 'other' | 610 products missing category |
| Non-delivered orders | orders | Filter to delivered | Analyzing completed transactions (96,478 of 99,441) |

In [ ]:
# Summary of cleaned dataframes available for analysis
print("Cleaned DataFrames Summary:")
print("="*50)
print(f"orders:           {len(orders):,} rows (timestamps converted)")
print(f"orders_delivered: {len(orders_delivered):,} rows (delivered only)")
print(f"order_items:      {len(order_items):,} rows")
print(f"order_payments:   {len(order_payments):,} rows")
print(f"order_reviews:    {len(order_reviews):,} rows (nulls preserved)")
print(f"products:         {len(products):,} rows (nulls filled)")
print(f"customers:        {len(customers):,} rows")
print(f"sellers:          {len(sellers):,} rows")
print(f"geolocation_clean:{len(geolocation_clean):,} rows (deduplicated)")
print(f"product_category_translation: {len(product_category_translation):,} rows")

In [ ]:
# Keep connection open for feature engineering section
print("Data cleaning complete. Proceeding to feature engineering...")

---

## Feature Engineering

Now that data is cleaned, we create derived features that add business value for downstream analysis.

### Order-Level Features

Create delivery performance and time-based features for order analysis.

In [ ]:
# Delivery time features
# Calculate actual delivery days (from purchase to delivery)
orders['delivery_days'] = (
    pd.to_datetime(orders['order_delivered_customer_date']) - 
    pd.to_datetime(orders['order_purchase_timestamp'])
).dt.days

# Calculate estimated delivery days (from purchase to estimated delivery)
orders['estimated_delivery_days'] = (
    pd.to_datetime(orders['order_estimated_delivery_date']) - 
    pd.to_datetime(orders['order_purchase_timestamp'])
).dt.days

# Delivery delta: positive = late, negative = early
orders['delivery_delta'] = orders['delivery_days'] - orders['estimated_delivery_days']

# Binary flag for late deliveries (use 'Int64' for nullable integer to handle NaN)
orders['is_late'] = (orders['delivery_delta'] > 0).astype('Int64')

print("Delivery features created:")
print(f"  delivery_days: {orders['delivery_days'].notna().sum():,} non-null values")
print(f"  estimated_delivery_days: {orders['estimated_delivery_days'].notna().sum():,} non-null values")
print(f"  delivery_delta: {orders['delivery_delta'].notna().sum():,} non-null values")
print(f"  is_late: {orders['is_late'].sum():,} late deliveries ({orders['is_late'].mean()*100:.1f}%)")

In [ ]:
# Time-based features for seasonality and trend analysis
orders['order_month'] = pd.to_datetime(orders['order_purchase_timestamp']).dt.to_period('M')
orders['order_dow'] = pd.to_datetime(orders['order_purchase_timestamp']).dt.day_name()
orders['order_hour'] = pd.to_datetime(orders['order_purchase_timestamp']).dt.hour

print("Time-based features created:")
print(f"\nOrders by day of week:")
print(orders['order_dow'].value_counts())
print(f"\nOrders by month (sample):")
print(orders['order_month'].value_counts().head(10))

### Customer-Level Features (RFM Analysis)

RFM (Recency, Frequency, Monetary) analysis is a fundamental customer segmentation technique:
- **Recency**: Days since last purchase (lower = better)
- **Frequency**: Number of unique orders (higher = better)
- **Monetary**: Total spend (higher = better)

In [ ]:
# Merge orders with customers to get customer_unique_id
orders_customers = orders.merge(customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

# Merge with payments to get monetary value
orders_payments = orders_customers.merge(
    order_payments.groupby('order_id')['payment_value'].sum().reset_index(),
    on='order_id',
    how='left'
)

print(f"Merged dataset: {len(orders_payments):,} rows")
print(f"Unique customers: {orders_payments['customer_unique_id'].nunique():,}")

In [ ]:
# Calculate RFM metrics per customer
snapshot_date = pd.to_datetime(orders_payments['order_purchase_timestamp']).max()
print(f"Snapshot date (most recent order): {snapshot_date}")

rfm = orders_payments.groupby('customer_unique_id').agg(
    recency=('order_purchase_timestamp', lambda x: (snapshot_date - pd.to_datetime(x).max()).days),
    frequency=('order_id', 'nunique'),
    monetary=('payment_value', 'sum')
).reset_index()

print(f"\nRFM DataFrame created: {len(rfm):,} customers")
print(f"\nRFM Summary Statistics:")
print(rfm[['recency', 'frequency', 'monetary']].describe())

In [ ]:
# Merge RFM with customer location data
customers_with_rfm = customers.merge(rfm, on='customer_unique_id', how='left')

print(f"Customers with RFM: {len(customers_with_rfm):,} rows")
print(f"Customers with RFM data: {customers_with_rfm['recency'].notna().sum():,}")
print(f"\nSample data:")
customers_with_rfm[['customer_unique_id', 'customer_city', 'customer_state', 'recency', 'frequency', 'monetary']].head(10)

### Product-Level Features

Create price tiers and freight ratio features for product analysis.

In [ ]:
# Price tier categorization
order_items['price_tier'] = pd.cut(
    order_items['price'],
    bins=[0, 50, 150, 500, float('inf')],
    labels=['Budget', 'Mid-Range', 'Premium', 'Luxury']
)

print("Price tier distribution:")
print(order_items['price_tier'].value_counts())
print(f"\nPercentage breakdown:")
print((order_items['price_tier'].value_counts(normalize=True) * 100).round(1))

In [ ]:
# Freight ratio: shipping cost as percentage of product price
# Handle division by zero for free items
order_items['freight_ratio'] = order_items['freight_value'] / order_items['price'].replace(0, float('nan'))

print("Freight ratio statistics:")
print(order_items['freight_ratio'].describe())
print(f"\nItems with freight ratio > 50% (shipping costs more than half the product):")
high_freight = order_items[order_items['freight_ratio'] > 0.5]
print(f"  Count: {len(high_freight):,} ({len(high_freight)/len(order_items)*100:.1f}%)")

---

## Save Processed Data

Export all cleaned and feature-engineered DataFrames to CSV files for downstream analysis.

In [ ]:
import os

# Define output directory
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

# Save orders with engineered features
# Convert period to string for CSV compatibility
orders_to_save = orders.copy()
orders_to_save['order_month'] = orders_to_save['order_month'].astype(str)
orders_to_save.to_csv(f'{output_dir}orders_processed.csv', index=False)
print(f"Saved: orders_processed.csv ({len(orders_to_save):,} rows)")

# Save order items with engineered features
order_items.to_csv(f'{output_dir}order_items_processed.csv', index=False)
print(f"Saved: order_items_processed.csv ({len(order_items):,} rows)")

# Save customers with RFM features
customers_with_rfm.to_csv(f'{output_dir}customers_with_rfm.csv', index=False)
print(f"Saved: customers_with_rfm.csv ({len(customers_with_rfm):,} rows)")

# Save deduplicated geolocation
geolocation_clean.to_csv(f'{output_dir}geolocation_clean.csv', index=False)
print(f"Saved: geolocation_clean.csv ({len(geolocation_clean):,} rows)")

## Feature Engineering Summary

| Feature | DataFrame | Description | Business Value |
|---------|-----------|-------------|----------------|
| `delivery_days` | orders | Actual days from purchase to delivery | Measure logistics performance |
| `estimated_delivery_days` | orders | Promised delivery time | Set customer expectations |
| `delivery_delta` | orders | Difference between actual and estimated | Identify delivery issues |
| `is_late` | orders | Binary flag for late deliveries | Track SLA compliance |
| `order_month` | orders | Month-year of purchase | Seasonality analysis |
| `order_dow` | orders | Day of week | Weekly pattern analysis |
| `order_hour` | orders | Hour of purchase | Daily pattern analysis |
| `recency` | customers_with_rfm | Days since last purchase | Customer engagement |
| `frequency` | customers_with_rfm | Number of orders | Customer loyalty |
| `monetary` | customers_with_rfm | Total spend | Customer value |
| `price_tier` | order_items | Budget/Mid-Range/Premium/Luxury | Product segmentation |
| `freight_ratio` | order_items | Shipping cost as % of price | Pricing strategy insights |

In [ ]:
# Final summary of processed files
print("="*60)
print("DATA CLEANING & FEATURE ENGINEERING COMPLETE")
print("="*60)
print(f"\nProcessed files saved to: {output_dir}")
print(f"\nFiles created:")
print(f"  1. orders_processed.csv       - Orders with delivery & time features")
print(f"  2. order_items_processed.csv  - Items with price tier & freight ratio")
print(f"  3. customers_with_rfm.csv     - Customers with RFM metrics")
print(f"  4. geolocation_clean.csv      - Deduplicated geolocation data")
print(f"\nReady for exploratory data analysis (EDA) in notebook 02.")

In [ ]:
# Close database connection
conn.close()
print("Database connection closed.")